In [22]:
import argparse
import json
from conllu import parse
import pandas as pd
from conllu import parse_incr
import pickle
import numpy as np
import re
import os
import unicodedata
import numpy as np
from glob import glob
import operator as op
import warnings 
from owlready2 import * 
import random
warnings.filterwarnings('ignore')

## Funções para filtragem do conllu

In [2]:
#utils

def get_start(x, offset):
    start = str(int(x["start_char"]) - offset)
    return start
def get_end(x, offset):
    end = str(int(x["end_char"]) - offset)
    return end
def correct_start_end(df):
    offset = int(df.iloc[0]["misc"].get("start_char"))
    df['start'] = df['misc'].apply(lambda x: get_start(x, offset))
    df['end'] = df['misc'].apply(lambda x: get_end(x, offset))
    return df

def remove_null_tokens(df):#retira tokens nulos na sentenca
    df = df[df['misc'].notna()]
    df = df.reset_index(drop=True)
    df = df[df['deps'].notna()]
    df = df.reset_index(drop=True)
    return df

def ents_uris_counter_prefiltering(sentence,ents,uris): #contagem de entidades e uris pre-filtragem
    entidades_sentenca_old, uris_sentenca_old = ents, uris
    ents_list, uris_list = ents, uris
    for idx_row in range(len(sentence)):
        token = sentence.iloc[idx_row]['deps']
        grafo = sentence.iloc[idx_row]['misc'].get('grafo')
        if "B=" in token:
            ents_list.append(token)
        if grafo:
            uris_list.append(grafo)     
    return ents_list, uris_list

def verify_sentence_is_functioning(df_sentence):
    if df_sentence.iloc[0]["misc"].get('eliminar') == '':#retirar linha com eliminar
        print('eliminada')
        return 0
    for idx_row in range(len(df_sentence)):
        if df_sentence.iloc[idx_row]['misc'].get('start_char') == None:#problema com start_char
            print('start_char com problema')
            return 0
    return 1

def get_text_sentence(df):
    size_sentence = int(df_sentence.iloc[-1]["end"])
    text = " "*size_sentence
    for index, row in df_sentence.iterrows():
        text = text[:int(row["start"])] + row["form"] +text[int(row["end"]):]
    return text 

def get_word_join(df, index):
    entity = df.iloc[index]['deps']
    entity_I = entity.replace("B=","I=")
    count = 1
    word_join = ""
    row_main = df.iloc[index]
    start_word, end_word = row_main['start'], row_main['end'] 
    word_join = " ".join([word_join, row_main['form']])
    while index+count != len(df) and (df.iloc[index+count]["deps"] == entity_I or check_I_entities(df, index+count,entity_I)):
        row = df.iloc[index+count]
        word_join = " ".join([word_join, row['form']])
        end_word = row['end']
        count+=1
    return word_join, start_word, end_word

def check_I_entities(df, i,entity):
    #verifica se ainda está dentro da entidade usando o sistema BIO de tokens
    next_entity_is_I = (df.iloc[i]["deps"] == entity) or (df.iloc[i]["deps"] == None and df.iloc[i+1]["deps"] == entity)
    return next_entity_is_I

def check_sentence_for_pair_uris(df_sentence, ents, uris):
    df_sent_new = pd.DataFrame()
    ents_list_old, uris_list_old = ents, uris
    ents_list, uris_list = ents, uris
    
    countURIs = 0
    for idx_row in range(len(df_sentence)):
        token = df_sentence.iloc[idx_row]['deps']
        grafo = df_sentence.iloc[idx_row]['misc'].get('grafo')
        if 'B=' in token and grafo:
            countURIs+=1
            ents_list = ents_list + [token]
            uris_list = uris_list + [grafo]
            
            df_sentence.loc[idx_row,'grafo'] = grafo
            df_sentence.loc[idx_row,'text'] = get_text_sentence(df_sentence)
            word_join, start_word_join, end_word_join = get_word_join(df_sentence, idx_row)
            df_sentence.loc[idx_row,'word_join'] = word_join
            df_sentence.loc[idx_row,'word_join_start'] = start_word_join
            df_sentence.loc[idx_row,'word_join_end'] = end_word_join
            df_sentence.loc[idx_row,'index_e'] = int(idx_row)
            
            df_sent_new = df_sent_new.append(df_sentence.iloc[idx_row],ignore_index = True)
#             raise SystemExit("Stop right there!")   
    if countURIs > 1:
        return 1, df_sent_new, ents_list, uris_list
    return 0, df_sentence, ents_list_old, uris_list_old,

## Ler conllu para filtrar sentencas que contenham pelo menos duas URIs

In [46]:
CONLLU_PATH = "petroner-uri-2023-04-05.conllu"
data_file = open(CONLLU_PATH, "r", encoding="utf-8")
sentences=[]
for tokenlist in parse_incr(data_file):
    sentences.append(tokenlist)
print('Total number of sentences in conllu ->',len(sentences))
#algum problema nessas sentencas em diferentes petroner
sentences_with_issues = [12408, 13636, 15264, 21023, 21122, 23920, 24017]
# sentences_with_issues = [1749, 4747, 4935, 5066, 5235, 5798, 6802, 7858, 8881, 9271, 10689, 10691, 10695, 10750, 
#                          11144, 12408, 13393, 13636, 15264, 17494, 18462, 18476, 19346, 21023, 21105, 21122, 
#                         22163, 23920, 24017]

Total number of sentences in conllu -> 24035


## Rotina para filtrar o arquivo conllu (é necessário rodar)

In [52]:
df_new = pd.DataFrame()
contSentences = 0
ents, uris = [], []
for idxSentence in range(0,len(sentences)):
    if idxSentence not in sentences_with_issues:
        print('sentence = ' ,idxSentence)
        sentence = sentences[idxSentence]
        json_sent = json.dumps(sentence)
        df_sentence = pd.read_json(json_sent)
        df_sentence = remove_null_tokens(df_sentence) 
        if verify_sentence_is_functioning(df_sentence):
            df_sentence = correct_start_end(df_sentence)    
            
            checkQtdTokenURI, df_test, ents, uris = check_sentence_for_pair_uris(df_sentence,ents,uris)
            if checkQtdTokenURI == 1:
                contSentences+= 1
                df_test['sentence'] = contSentences
                df_test['#sentence_original'] = idxSentence
                df_new = df_new.append(df_test)
                       
                print('sentences after filtering ->', contSentences)
#                 raise SystemExit("Stop right there!")

df_new.drop(columns=["start","end","head","id","feats","upos","lemma","xpos"], inplace=True)

pickle.dump(df_new, open('df_filtred_petroner_uri_2022_15_05.conllu.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
df_new.to_csv('df_filtred_petroner_uri_2022_15_05_conllu.csv',encoding = 'utf-8',index=False)

#entidades é a lista de diferentes tipos de entidades que aparecem após filtragem das sentencas
entidades, numb_ents = np.unique(ents, return_counts = True)
#grafos é a lista de diferentes URIs anotadas que aparecem após filtragem das sentencas
grafos, numb_grafos = np.unique(uris, return_counts = True)
print('------------')
print('lista de diferentes tipos de entidades pós filtragem')
print(entidades.tolist())
print('------------')
print('quantidades de diferentes tipos de entidades pós filtragem')
print(numb_ents.tolist())
print('------------')
print('Total de URIs pós filtragem ->', sum(numb_grafos.tolist()))

sentence =  0
sentence =  1
sentence =  2
sentence =  3
sentence =  4
sentence =  5
sentence =  6
sentences after filtering -> 1
sentence =  7
sentence =  8
sentence =  9
sentence =  10
sentence =  11
sentence =  12
sentence =  13
sentence =  14
sentence =  15
sentence =  16
sentence =  17
sentence =  18
sentence =  19
sentence =  20
sentence =  21
sentence =  22
sentence =  23
sentence =  24
sentence =  25
sentence =  26
sentence =  27
sentence =  28
sentence =  29
sentence =  30
sentence =  31
sentence =  32
sentence =  33
eliminada
sentence =  34
sentence =  35
eliminada
sentence =  36
sentences after filtering -> 2
sentence =  37
sentence =  38
sentence =  39
sentence =  40
sentence =  41
sentence =  42
sentence =  43
sentence =  44
sentence =  45
sentence =  46
sentence =  47
sentence =  48
sentence =  49
sentences after filtering -> 3
sentence =  50
sentence =  51
sentence =  52
sentence =  53
sentence =  54
sentences after filtering -> 4
sentence =  55
sentence =  56
sentence = 

## Rotina para contar entidades e URIs no conllu (não é necessário rodar)

In [172]:
ents_pre = []
uris_pre = []
for idxGroups in range(0,len(sentences)):
    if idxGroups not in sentences_with_issues: 
        print('sentence = ' ,idxGroups)
        sentence = sentences[idxGroups]
        json_temp = json.dumps(sentence)
        df_get_start_end = pd.read_json(json_temp)
        df_get_start_end = remove_null_tokens(df_get_start_end) 
        if verify_sentence_is_functioning(df_get_start_end):
            df_get_start_end = correct_start_end(df_get_start_end) 
        
            ents_pre, uris_pre = ents_uris_counter_prefiltering(df_get_start_end,ents_pre,uris_pre)
        
entidades_pre, numb_ents_pre = np.unique(ents_pre, return_counts = True)
grafos_pre, numb_grafos_pre = np.unique(uris_pre, return_counts = True)
print('------------')
print('lista de diferentes tipos de entidades no conllu')
print(entidades_pre.tolist())
print('------------')
print('quantidades de diferentes tipos de entidades no conllu')
print(numb_ents_pre.tolist())
print('------------')
print('Total de URIs no Conllu original ->',sum(numb_grafos_pre.tolist()))

sentence =  0
sentence =  1
sentence =  2
sentence =  3
sentence =  4
sentence =  5
sentence =  6
sentence =  7
sentence =  8
sentence =  9
sentence =  10
sentence =  11
sentence =  12
sentence =  13
sentence =  14
sentence =  15
sentence =  16
sentence =  17
sentence =  18
sentence =  19
sentence =  20
sentence =  21
sentence =  22
sentence =  23
sentence =  24
sentence =  25
sentence =  26
sentence =  27
sentence =  28
sentence =  29
sentence =  30
sentence =  31
sentence =  32
sentence =  33
ELIMINADA
sentence =  34
sentence =  35
ELIMINADA
sentence =  36
sentence =  37
sentence =  38
sentence =  39
sentence =  40
sentence =  41
sentence =  42
sentence =  43
sentence =  44
sentence =  45
sentence =  46
sentence =  47
sentence =  48
sentence =  49
sentence =  50
sentence =  51
sentence =  52
sentence =  53
sentence =  54
sentence =  55
sentence =  56
sentence =  57
sentence =  58
sentence =  59
sentence =  60
sentence =  61
sentence =  62
sentence =  63
sentence =  64
sentence =  65


# Rotina para processar as sentenças já filtradas

## Carregar ontologia

In [23]:
onto_name = "OntoGeoLogicaPovoadaInstanciasRelacoes"
onto = get_ontology("./ner_geologica/OntoGeoLogicaPovoadaInstanciasRelacoes.owl")
onto.load()

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#has_beginning

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#has_end

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_contains

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_during

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_in

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#interval_finished_by

* Owl

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

## Funções para procurar na ontologia

In [24]:
def get_relations_between_uris(uri_1, uri_2): 
    #funcao que acessa a ontologia e procura relacao entre URIs
    dict_relation_uris = {}
    #Pega as relacoes que a URI1 tem
    relation_query_results = list(default_world.sparql("""
            SELECT DISTINCT ?rel
            WHERE{?uri ?rel ?obj
                 FILTER(contains(str(?rel), "http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#"))
                 FILTER (contains(str(?uri), """ + '"' + uri_1 + '"' + """))
                 }
            """))
    
    relations_str = []
    for relation_uris in relation_query_results:
        relations_str.append(str(relation_uris[0]).rsplit(".",1)[-1])
        
    # Para cada tipo de relação procura se existe match entre URI1 e URI2
    for relation in relations_str:
        relation_between_words = list(default_world.sparql("""
                PREFIX prefix: <http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#>
                SELECT distinct ?y ?x2
                WHERE{?y prefix:""" +  relation  +  """ ?x1

                      FILTER (contains(str(?y), """ + '"' + uri_1  + '"' + """))        

                      ?x2 rdf:type ?j                                   
                      FILTER (contains(str(?x2), """ + '"' + uri_2  + '"' + """))

                      FILTER ( ?x2 = ?x1 )
                    }
                """))
        dict_relation_uris[relation] = relation_between_words
    return dict_relation_uris

def go_through_relations(uri1,uri2):
    relation_uris = get_relations_between_uris(uri1, uri2)            
    if relation_uris != {}: #talvez exista relacao entre URIs, dicionario pode vir vazio -> []
        for x, y in relation_uris.items():#procurar por relacao
            if y != []: #existe alguma relacao
#                 print(x)
                return x
    return None

### Funções para printar informações

In [25]:
def print_sentence_text(sentence):
    #printa e retorna o texto original da sentenca
    size_sentence = int(sentence.iloc[-1]["end"])
    text = " "*size_sentence
    for index, row in sentence.iterrows():
        text = text[:int(row["start"])] + row["form"] +text[int(row["end"]):]
    print(text)
    print("-------------")
    return text

def print_relation_entities(word1,word2,ent1,ent2,URI_1,URI_2,relation_type,text):
    #printa as entidades e relacao entre elas
    print('Token 1 = ', word1, '--- Class 1 = ', ent1, '--- URI 1 = ', URI_1)
    print('Token 2 = ', word2, '--- Class 2 = ', ent2,'--- URI 2 = ',URI_2)
    print('Relation Type = ', relation_type)
    print(text)
    print("-------------")
    pass

## Funções para gerar Jsons a serem lidos no labelstudio

In [26]:
class ResultRelationJson(object):
    def __init__(self, from_id, to_id, relations, direction = "right"):
        self.dict = {
            "from_id": str(from_id),
            "to_id": str(to_id),
            "type": "relation",
            "direction": direction,
            "labels": relations
        }
    def get_dict(self):
        return self.dict
class ResultNERJson(object):
    def __init__(self, row):     
        self.result_dict = {
            "value": {
            "start": row["start_word"],
            "end": row["end_word"],
            "text": row["word_join"],
            "labels": [
              row["label_word"]
            ],
            "URI": row["URI"]
            },
            
            "id": row["index_e"],
            "from_name": "label",
            "to_name": "text",
            "type": "labels",
            "origin": "prediction"
        }
    def get_dict(self):
        return self.result_dict  
class CreateOutput(object):
    def __init__(self, text, filtred_sentence, entity_name_new):
        self.filtred_sentence = filtred_sentence
        self.entity_name_new = entity_name_new
        self.main_dict = {
            "id": 1,
            "data": {
              "text": text #sentenca inteira
            },
            "annotations": []
        }
        self._add_annotations()      
    def _add_annotations(self):
        results = []
        count = 0        
        for index, row in self.entity_name_new.iterrows(): 
            results.append(ResultNERJson(row).get_dict())        
        item = [{
              "id": 1,
              "created_username": " null, 0",
              "created_ago": "",
              "result": results
            }]
        self.main_dict["annotations"] = item
    def get_output(self):
        return self.main_dict
    def add_relationship(self, from_id, to_id, relations, direction):
        results = self.main_dict.get("annotations")[0].get("result")
        relation = ResultRelationJson(from_id, to_id, [relations], direction).get_dict()
        results.append(relation)
        self.main_dict["annotations"][0]["result"] = results   
        
def combine_itens_from_lists_add_in_json(from_id_vec, to_id_vec, relation_from_vec, output):
    for idxRelation in range(0,len(from_id_vec)):
        direction = "right"
        output.add_relationship(from_id=from_id_vec[idxRelation], to_id=to_id_vec[idxRelation], relations = relation_from_vec[idxRelation], direction=direction)
    return output
def saveJsonFiles(df,from_id,to_id, lista_relaoces_sentence,sentence,SentenceNum,path):
    #cria e salva o arquivo Json para labelstudio
    text = sentence.iloc[0]['text']
    print('Saved Json ->', True)
    output = CreateOutput(text,sentence, df)
    combine_itens_from_lists_add_in_json(from_id, to_id, lista_relacoes_sentence, output)
    print("-------------")
    with open(os.path.join(path,f"{SentenceNum}.json"), "w") as outfile: 
        json.dump(output.get_output(), outfile) 
        
def get_df_forJsons(sentence,idxTokens):
    #retorna um dataframe com as informações das entidades e uma string contendo o nome completo da entidade
    df_save_words = pd.DataFrame(columns=['index_e', "LABEL", "START", "END",\
                                      "TEXT", "word_join", "start_word", "end_word", "label_word","URI"])

    index_e = sentence.iloc[idxTokens]['index_e']
    label = sentence.iloc[idxTokens]['deps']
    start = sentence.iloc[idxTokens]['word_join_start']
    end = start + len(sentence.iloc[idxTokens]['form'])
    text_ent = sentence.iloc[idxTokens]['form']
    word_join = sentence.iloc[idxTokens]['word_join']
    start_word = sentence.iloc[idxTokens]['word_join_start']
    end_word = sentence.iloc[idxTokens]['word_join_end']
    label_word = label.replace("B=","")
    URI = sentence.iloc[idxTokens]['grafo']

    df_save_words.loc[len(df_save_words.index)] = [index_e, label, start, end, text_ent, word_join,
                                                   start_word,
                                                   end_word,
                                                   label_word,
                                                   URI]

    return df_save_words, word_join
        
def create_df_JsonFiles(df_entity,x,token,token2,URI_1,URI_2,idxTokens,idxTokens2,from_id,to_id,sentence):
    #retorna o dataframe utilizado para criacao dos arquivos Json para labelstudio
    entity_name_new_token1,wordjoin_1 = get_df_forJsons(sentence,idxTokens)
    entity_name_new_token2,wordjoin_2 = get_df_forJsons(sentence,idxTokens2)
    if idxTokens not in from_id and idxTokens not in to_id:
        df_entity = pd.concat([df_entity, entity_name_new_token1])
    if idxTokens2 not in from_id and idxTokens2 not in to_id:
        df_entity = pd.concat([df_entity, entity_name_new_token2])

#     print_relation_entities(wordjoin_1,wordjoin_2,token.replace('B=',''),token2.replace('B=',''),URI_1,URI_2,x)
    return df_entity  

### Funções para criar dataframe para modelo BERT

In [27]:
def create_moddedText_BERT(text,start1,end1,start2,end2,Ent1_inic,Ent1_end,Ent2_inic,Ent2_end):
    new_end_ent1 = end1 + len(Ent1_inic)
    new_start_ent2 = start2 + len(Ent1_inic) + len(Ent1_end)
    new_end_ent2 = end2 + len(Ent1_inic) + len(Ent1_end) + len(Ent2_inic)      
    #adicionando [E1] e [/E1]
    text_new = text[:start1] + Ent1_inic + text[start1:]
    text_new = text_new[:new_end_ent1] + Ent1_end + text_new[new_end_ent1:]
    #adicionando [E2] e [/E2]
    text_new2 = text_new[:new_start_ent2] + Ent2_inic + text_new[new_start_ent2:]
    text_new2 = text_new2[:new_end_ent2] + Ent2_end + text_new2[new_end_ent2:]
    
    return text_new2

def createText_sentence_BERT(text,start_1,end_1,start_2,end_2):
    #funcao que retorna um novo texto para sentenca com [E1] e [E2] adicionados junto de cada entidade
    start_ent1, start_ent2 = start_1, start_2
    end_ent1, end_ent2 = end_1, end_2
    Ent1_inic, Ent1_end = '[E1] ', ' [/E1]'
    Ent2_inic, Ent2_end = '[E2] ', ' [/E2]'
    
    if start_ent1 < start_ent2: #[E1] vem antes de [E2]
        text_new = create_moddedText_BERT(text,start_ent1,end_ent1,start_ent2,end_ent2,\
                                  Ent1_inic,Ent1_end,Ent2_inic,Ent2_end)
    else: #[E2] vem antes de [E1]      
        text_new = create_moddedText_BERT(text,start_ent2,end_ent2,start_ent1,end_ent1,\
                                      Ent2_inic,Ent2_end,Ent1_inic,Ent1_end)  
    return text_new

def create_bert_dataframe(df_bert,idxTokens,idxTokens2,sentence,URI_1,URI_2,has_relation,relation_type,SentenceNumber):
    #retorna o dataframe com as informacoes de cada sentenca para utilizar no modelo BERT
    df_bert_temp = pd.DataFrame(columns=['#Sentence','sentence','Ent1','Ent2','URI_1','URI_2','has_relation','relation'])
    text = sentence.iloc[0]['text']
    wordjoin_1, wordjoin_2 = sentence.iloc[idxTokens]['word_join'], sentence.iloc[idxTokens2]['word_join']
    ent1, ent2 = sentence.iloc[idxTokens]['deps'], sentence.iloc[idxTokens2]['deps']
    ent1, ent2 = ent1.replace("B=",""), ent2.replace("B=","")
    start_1, start_2 = sentence.iloc[idxTokens]['word_join_start'], sentence.iloc[idxTokens2]['word_join_start']
    end_1, end_2 = sentence.iloc[idxTokens]['word_join_end'], sentence.iloc[idxTokens2]['word_join_end']
    text_bert_ents = createText_sentence_BERT(text,start_1,end_1,start_2,end_2)
    df_bert_temp.loc[0] = [SentenceNumber,
                           text_bert_ents,
                           ent1,
                           ent2,
                           URI_1,
                           URI_2,
                           has_relation,
                           relation_type]
    df_bert = pd.concat([df_bert, df_bert_temp])
    if relation_type!='no_relation':
        print_relation_entities(wordjoin_1,wordjoin_2,ent1,ent2,URI_1,URI_2,relation_type,text_bert_ents)
    return df_bert

## Funções para processar as sentenças

In [32]:
#utils
def create_relations_dataframe(df_relation,token,token2,URI_1,URI_2,x,originalSentenceNumber):
    #retorna dataframe das entidades e suas relações em cada linha
    #importante para contabilizar os tipos de relação
    df_relation_new = pd.DataFrame(columns=['Relation','Ent1','Ent2','URI_1','URI_2','#Sentence'])
    df_relation_new.loc[0] = [x,
                            token.replace('B=',''),
                            token2.replace('B=',''),
                            URI_1,
                            URI_2,
                            originalSentenceNumber]
    df_relation = pd.concat([df_relation, df_relation_new])
    return df_relation

def verifica_pares_entidade_interesse(ENT_1, ENT_2,relation_type):
    #verifica se a relacao encontrada vai ser do tipo temporal_relation CRONO->CRONO
    #funcao talvez precise ser atualizada no futuro conforme a ontologia for povoada
    lista_from = ['POÇO','UNIDADE_LITO','UNIDADE_LITO','CAMPO','POÇO','POÇO','UNIDADE_LITO','UNIDADE_LITO']
    lista_to = ['UNIDADE_LITO','NÂOCONSOLID','ROCHA','BACIA','BACIA','CAMPO','BACIA','UNIDADE_CRONO']        
    for idx in range(0,len(lista_to)):
        if lista_from[idx] == ENT_1 and lista_to[idx] == ENT_2:
            return relation_type
#     return 'temporal_relation'
    return relation_type

def go_through_sentence(sentence_df,df_relation,df_bert,sent_numb):
    #percorre a sentenca em busca de relacoes entre entidades anotadas com URIs
    df_entity = pd.DataFrame()
    from_id, to_id = [], []
    relation_from, relation_to = [], []
    lista_relacoes_sentence = []
    is_to_save = False
#     df_bert.to_csv(save_csv_name, encoding='utf-8',index=False)
    df_relation.to_csv('df_relation.csv', encoding='utf-8',index=False)
    for idxTokens in range(len(sentence_df)):
        token, URI_1 = sentence_df.iloc[idxTokens]['deps'], sentence_df.iloc[idxTokens]['grafo']
        for idxTokens2 in range(len(sentence_df)):
            if idxTokens != idxTokens2:
                token2, URI_2 = sentence_df.iloc[idxTokens2]['deps'], sentence_df.iloc[idxTokens2]['grafo']
                has_relation = False
                relation_type = go_through_relations(URI_1,URI_2)
                if relation_type: 
                    print("-------------")
                    print('sentence =', sent_numb)
                    is_to_save = True
                    has_relation = True
                    Ent1, Ent2 = token.replace("B=",""), token2.replace("B=","")
                    relation_type = verifica_pares_entidade_interesse(Ent1,Ent2,relation_type)
                    lista_relacoes_sentence.append(relation_type)

                    #criar df_bert para BERT RE com codigo do Fabio
                    df_bert = create_bert_dataframe(df_bert,idxTokens,idxTokens2,sentence_df,
                                                    URI_1,URI_2,
                                                    has_relation,relation_type,originalSentenceNumber)

                    #para contabilizar os pares de entidade por relacao
                    df_relation = create_relations_dataframe(df_relation,token,token2,
                                                             URI_1,URI_2,relation_type,originalSentenceNumber)
                    #listas para contabilizar relacoes, uris e classes
                    lista_relacoes.append(relation_type)
                    lista_uris.append(URI_1)
                    lista_uris.append(URI_2)         
                    lista_classes.append(Ent1)
                    lista_classes.append(Ent2)

                    if is_to_createJsons: #se quiser criar Jsons para LabelStudio
                        df_entity = create_df_JsonFiles(df_entity,relation_type,token,token2,URI_1,URI_2,
                                                        idxTokens,idxTokens2,from_id,to_id,sentence_df)
                        from_id.append(idxTokens)
                        to_id.append(idxTokens2) 

                else: #nao achou relacao
                    relation_type = 'no_relation'

                    df_bert = create_bert_dataframe(df_bert,idxTokens,idxTokens2,sentence_df,
                                                    URI_1,URI_2,has_relation,relation_type,originalSentenceNumber)
                        
    return lista_relacoes,lista_uris,lista_classes,\
            df_bert, df_relation, df_entity, \
            lista_relacoes_sentence, from_id, to_id, is_to_save

### Ler arquivo csv (ou pkl) com as sentenças pós filtragem

In [33]:
# df_filtred_sentences = pickle.load(open('df_filtred_petroner_uri_2023_15_05.conllu.pkl', 'rb'))
df_filtred_sentences = pd.read_csv('df_filtred_petroner_uri_2022_15_05_conllu.csv')
df_group = df_filtred_sentences.groupby('sentence')
print('Numero total de sentenças pos-filtragem -> ',len(df_group))

Numero total de sentenças pos-filtragem ->  4542


### Escolher se deseja criar Jsons para labelstudio

In [34]:
# is_to_createJsons = True
is_to_createJsons = False

### Folder outputs

In [35]:
save_folder_path = "./JSONs_15_05" #local onde são salvos os Jsons para labelstudio
save_csv_name = 'df_bert_sentences_15_05.csv'

## Rotina para processar as sentenças já filtradas 

In [ ]:
%%time

numberSentences = df_filtred_sentences.iloc[-1]['sentence'] #numero de sentencas diferentes no arquivo ja filtrado
lista_relacoes, lista_uris, lista_classes, list_sentences_dict = [], [], [], []
df_relation, df_bert = pd.DataFrame(), pd.DataFrame()

countJsons=0
for idx in range(1,len(df_group)):
    filtred_sentence = df_group.get_group(idx)#aqui filtred_sentence é um dataframe da sentenca
    originalSentenceNumber = filtred_sentence.iloc[0]['#sentence_original']
    text = filtred_sentence.iloc[0]['text']
    lista_relacoes,lista_uris,lista_classes,df_bert,\
    df_relation,df_entity,lista_relacoes_sentence,\
    from_id,to_id, is_to_save = go_through_sentence(filtred_sentence,df_relation,df_bert,originalSentenceNumber) 
#     df_bert.to_csv(save_csv_name, encoding='utf-8',index=False)
    if is_to_save and is_to_createJsons:#Jsons somente criados para sentencas com relacao
        countJsons+=1
        saveJsonFiles(df_entity,from_id,to_id, 
                      lista_relacoes_sentence,filtred_sentence,originalSentenceNumber,save_folder_path)
#     raise SystemExit("Stop right there!")    
print("-------------")
print("Number of Jsons saved = ", countJsons)

# pickle.dump(df_relation, open('df_relation.pkl','wb'), protocol=pickle.HIGHEST_PROTOCOL)
df_relation.to_csv('df_relation.csv', encoding='utf-8',index=False)
# df_bert.to_csv(save_csv_name, encoding='utf-8',index=False)

relacoes, numb_rel = np.unique(lista_relacoes, return_counts = True)
pickle.dump(lista_relacoes,open('lista_relacoes.pkl','wb'),protocol = pickle.HIGHEST_PROTOCOL)

uris, numb_uris = np.unique(lista_uris, return_counts = True)
pickle.dump(lista_uris,open('lista_uris.pkl','wb'),protocol = pickle.HIGHEST_PROTOCOL)

classes, numb_classes = np.unique(lista_classes, return_counts = True)
pickle.dump(lista_classes,open('lista_classes.pkl','wb'),protocol = pickle.HIGHEST_PROTOCOL)

-------------
sentence = 6
Token 1 =   Membro Mucuri --- Class 1 =  UNIDADE_LITO --- URI 1 =  #membro_010
Token 2 =   Bacia Espirito Santo --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_270
Relation Type =  located_in
[E1] Membro Mucuri [/E1], Eocretáceo    [E2] Bacia    Espirito Santo [/E2]..
-------------
-------------
sentence = 97
Token 1 =   Formação Pendência --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_319
Token 2 =   Bacia Potiguar --- Class 2 =  BACIA --- URI 2 =  #BASE_CD_BACIA_100
Relation Type =  located_in
2.2 - Estratigrafia de Sequências    [E1] Formação Pendência [/E1], [E2] Bacia Potiguar [/E2]  Della Favera et al. (1992) analisam a seção sin-rift    Bacia Potiguar,    qual identificaram quatro sequências balizadas por discordâncias e suas concordâncias relativas dentro    Formação Pendência..
-------------
-------------
sentence = 97
Token 1 =   Formação Pendência --- Class 1 =  UNIDADE_LITO --- URI 1 =  #formacao_319
Token 2 =   Bacia Potiguar --- Class 2 =  

### Contabilização das relações encontradas

In [33]:
relacoes.tolist()

['atravessa',
 'constituted_by',
 'has_age',
 'interval_finishes',
 'located_in',
 'temporal_relation']

In [34]:
numb_rel.tolist()

[29, 402, 75, 1, 526, 1034]

### Contabilização das classes encontradas

In [35]:
classes.tolist()

['BACIA',
 'CAMPO',
 'NÃOCONSOLID',
 'POÇO',
 'ROCHA',
 'UNIDADE_CRONO',
 'UNIDADE_LITO']

In [36]:
numb_classes.tolist()

[492, 162, 7, 118, 402, 2130, 823]

### Contabilização das URIs encontradas 

In [17]:
uris.tolist()

['#Alagoas_Age',
 '#Albian',
 '#Aptian',
 '#Aratu_Age',
 '#Archean',
 '#Atokan_Age',
 '#BASE_CD_BACIA_020',
 '#BASE_CD_BACIA_030',
 '#BASE_CD_BACIA_051',
 '#BASE_CD_BACIA_076',
 '#BASE_CD_BACIA_080',
 '#BASE_CD_BACIA_090',
 '#BASE_CD_BACIA_096',
 '#BASE_CD_BACIA_100',
 '#BASE_CD_BACIA_106',
 '#BASE_CD_BACIA_116',
 '#BASE_CD_BACIA_210',
 '#BASE_CD_BACIA_215',
 '#BASE_CD_BACIA_230',
 '#BASE_CD_BACIA_240',
 '#BASE_CD_BACIA_250',
 '#BASE_CD_BACIA_256',
 '#BASE_CD_BACIA_260',
 '#BASE_CD_BACIA_266',
 '#BASE_CD_BACIA_270',
 '#BASE_CD_BACIA_281',
 '#BASE_CD_BACIA_300',
 '#BASE_CD_BACIA_316',
 '#BASE_CD_BACIA_381',
 '#Barremian',
 '#Bartonian',
 '#Berriasian',
 '#Buracica_Age',
 '#Burdigalian',
 '#CAMP_CD_CAMPO_0003',
 '#CAMP_CD_CAMPO_0004',
 '#CAMP_CD_CAMPO_0012',
 '#CAMP_CD_CAMPO_0017',
 '#CAMP_CD_CAMPO_0027',
 '#CAMP_CD_CAMPO_0065',
 '#CAMP_CD_CAMPO_0077',
 '#CAMP_CD_CAMPO_0082',
 '#CAMP_CD_CAMPO_0093',
 '#CAMP_CD_CAMPO_0118',
 '#CAMP_CD_CAMPO_0174',
 '#CAMP_CD_CAMPO_0179',
 '#CAMP_CD_CAMPO_

In [18]:
numb_uris.tolist()

[51,
 140,
 127,
 26,
 6,
 40,
 2,
 12,
 3,
 2,
 1,
 2,
 35,
 48,
 4,
 111,
 1,
 4,
 11,
 55,
 8,
 1,
 9,
 26,
 27,
 75,
 28,
 22,
 4,
 18,
 2,
 2,
 42,
 4,
 1,
 1,
 1,
 5,
 1,
 4,
 4,
 6,
 1,
 2,
 1,
 1,
 1,
 4,
 4,
 1,
 2,
 1,
 3,
 2,
 1,
 1,
 1,
 13,
 2,
 2,
 2,
 4,
 1,
 1,
 1,
 4,
 1,
 4,
 3,
 3,
 2,
 2,
 1,
 1,
 1,
 8,
 2,
 2,
 1,
 1,
 6,
 1,
 1,
 12,
 1,
 1,
 1,
 1,
 6,
 4,
 1,
 1,
 1,
 3,
 1,
 1,
 3,
 3,
 3,
 265,
 138,
 49,
 47,
 36,
 19,
 32,
 92,
 11,
 16,
 41,
 22,
 18,
 32,
 2,
 3,
 14,
 53,
 41,
 31,
 113,
 3,
 2,
 22,
 12,
 2,
 2,
 7,
 123,
 2,
 40,
 3,
 6,
 12,
 6,
 8,
 24,
 26,
 8,
 2,
 2,
 21,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 1,
 6,
 4,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 2,
 1,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 48,
 17,
 5,
 65,
 5,
 60,
 1,
 2,
 56,
 6,
 2,
 13,
 15,
 11,
 14,
 60,
 6,
 2,
 2,
 4,


### Verificação de pares de entidades por tipo de relação

In [ ]:
# df_filtred = pickle.load(open('df_relation.pkl', 'rb'))
df_relations = pd.read_csv('df_relation.csv')
df_grp = df_relations.groupby('Relation')
relations_groups = df_grp.groups
relations = list(relations_groups)
lista_pares = []
for relation in relations:
    df_rel = df_grp.get_group(relation)
    list_rel = []
    for idx_rel in range(0,len(df_rel)):
        par = df_rel.iloc[idx_rel]['Ent1'] + ' + ' + df_rel.iloc[idx_rel]['Ent2']
        list_rel.append(par)
    lista_pares.append(list_rel)
print('Number of types of relations ->', len(lista_pares))

### Avaliar idx_pair de 0 ao tamanho apresentado acima para verificar os pares de entidades

In [ ]:
idx = 0
pares, numb_pares = np.unique(lista_pares[idx], return_counts = True)
print('Relation -> ',relations[idx])
print('Entities pair -> ',pares.tolist())
print('Number of ocorrences -> ',numb_pares.tolist())

In [2]:
df_filtred_sentences = pd.read_csv('df_bert_sentences_15_05.csv')

In [20]:
df_filtred_sentences.iloc[86:89]

,#Sentence,sentence,Ent1,Ent2,URI_1,URI_2,has_relation,relation
86,97,2.2 - Estratigrafia de Sequências [E1] Form...,UNIDADE_LITO,BACIA,#formacao_319,#BASE_CD_BACIA_100,True,located_in
87,97,2.2 - Estratigrafia de Sequências [E1] Form...,UNIDADE_LITO,BACIA,#formacao_319,#BASE_CD_BACIA_100,True,located_in
88,97,2.2 - Estratigrafia de Sequências [E1] Form...,UNIDADE_LITO,ESTRUTURA_FÍSICA,#formacao_319,#TEFR_CD_TIPO_EST_FISICA_ROCHA_036,False,no_relation


In [21]:
df_filtred_sentences.iloc[87]['sentence']

'2.2 - Estratigrafia de Sequências    [E1] Formação Pendência [/E1], Bacia Potiguar  Della Favera et al. (1992) analisam a seção sin-rift    [E2] Bacia Potiguar [/E2],    qual identificaram quatro sequências balizadas por discordâncias e suas concordâncias relativas dentro    Formação Pendência..'